## in this script we're aggregating cases from zipcode into PUMA and then into KC region to be prepped for MASCOT GLM

In [28]:
import pandas as pd
import xlrd
import numpy as np

In [29]:
kc_data = '/Users/miguelparedes/Downloads/positives_ZIP_daily-mar-28.csv' #data taken from https://kingcounty.gov/depts/health/covid-19/data/daily-summary.aspx

In [30]:
dfnew = pd.read_csv(kc_data, parse_dates = [2])
dfnew['Positives'] = dfnew['Positives'].astype('int')
dfnew.Positive_Rate[dfnew["Positive_Rate"] == "#NUM!"] = 0
dfnew['Positive_Rate'] = dfnew['Positive_Rate'].astype('float')
dfnew['ZIP'] = dfnew['ZIP'].astype('int')

/Users/miguelparedes/anaconda3/envs/nextstrain/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
# go from zipcode to PUMA
dfnew['PUMA'] = np.nan
for index, rows in dfnew.iterrows():
    if rows.ZIP in (98103, 98107, 98117):
        dfnew.loc[index, 'PUMA'] = "11601" 
    elif rows.ZIP in (98105, 98115, 98125, 98195):
        dfnew.loc[index, 'PUMA'] = "11602" 
    elif rows.ZIP in (98101, 98102, 98104, 98109, 98119, 98121, 98154, 98164, 98199):
        dfnew.loc[index, 'PUMA'] = "11603" 
    elif rows.ZIP in (98112, 98118, 98122, 98144):
        dfnew.loc[index, 'PUMA'] = "11604" 
    elif rows.ZIP in (98106, 98108, 98116, 98126, 98134, 98136):
        dfnew.loc[index, 'PUMA'] = "11605" 
    elif rows.ZIP in (98133, 98155, 98177, 98011, 98028):
        dfnew.loc[index, 'PUMA'] = "11606" 
    elif rows.ZIP in (98033, 98034, 98052):
        dfnew.loc[index, 'PUMA'] = "11607" 
    elif rows.ZIP in (98004, 98005, 98006, 98007, 98008, 98039):
        dfnew.loc[index, 'PUMA'] = "11608" 
    elif rows.ZIP in (98040, 98029, 98074, 98075):
        dfnew.loc[index, 'PUMA'] = "11609" 
    elif rows.ZIP in (98055, 98057, 98056, 98178):
        dfnew.loc[index, 'PUMA'] = "11610" 
    elif rows.ZIP in (98146, 98148, 98166, 98168, 98188):
        dfnew.loc[index, 'PUMA'] = "11611" 
    elif rows.ZIP in (98003, 98023, 98198, 98070):
        dfnew.loc[index, 'PUMA'] = "11612" 
    elif rows.ZIP in (98030, 98031, 98032, 98092):
        dfnew.loc[index, 'PUMA'] = "11613" 
    elif rows.ZIP in (98001, 98002, 98047):
        dfnew.loc[index, 'PUMA'] = "11614" 
    elif rows.ZIP in (98010, 98022, 98038, 98051, 98027, 98042, 98059, 98058):
        dfnew.loc[index, 'PUMA'] = "11615" 
    elif rows.ZIP in (98045, 98065, 98014, 98077, 98053, 98024, 98072, 98019):
        dfnew.loc[index, 'PUMA'] = "11616" 

In [35]:
#go from PUMA to KC region
north_kc = ["11601", "11602", "11603", "11604", "11605", "11606", "11607", "11608", "11609",  "11616" ]
south_kc = [ "11610", "11611", "11612", "11613", "11614", "11615"]
dfnew['region'] = np.nan
for index, row in dfnew.iterrows():
    if row.PUMA in north_kc:
        dfnew.loc[index, "region"] = "nKC"
    elif row.PUMA in south_kc:
        dfnew.loc[index, "region"] = "sKC"
    

In [37]:
#14 rolling average day average for hospitalization by region
dfnew['Date'] = pd.to_datetime(dfnew['Collection_Date']) 
df_test = dfnew.groupby(['region', pd.Grouper(key='Date')])['Positives'].sum().rolling(14, min_periods=1).mean()
df = df_test.reset_index()
df


,region,Date,Positives
0,nKC,2020-02-01,0.000000
1,nKC,2020-02-02,0.000000
2,nKC,2020-02-03,0.000000
3,nKC,2020-02-04,0.000000
4,nKC,2020-02-05,0.000000
...,...,...,...
1567,sKC,2022-03-23,50.071429
1568,sKC,2022-03-24,49.571429
1569,sKC,2022-03-25,51.285714
1570,sKC,2022-03-26,51.642857


In [38]:
#df.to_csv("../data-files/cases_per_100_by_region.csv")

In [39]:
#MASCOT will log normalize these values and thus we can't have any 0 values. 
df.Positives[df.Positives <= 0] = 0.1

/Users/miguelparedes/anaconda3/envs/nextstrain/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
nkc_df = df[df['region'] == "nKC"]
skc_df = df[df['region'] == "sKC"]

skc_df = skc_df[['region', 'Date', 'Positives']]
nkc_df = nkc_df[['region', 'Date', 'Positives']]

skc_df = skc_df.rename(columns = {"Positives": "Positives_skc"}).reset_index()
nkc_df = nkc_df.rename(columns = {"Positives": "Positives_nkc"}).reset_index()

In [45]:
#merging n and s datasets together
merged_df = skc_df.merge(nkc_df, on = "Date")

In [46]:
merged_df

,index_x,region_x,Date,Positives_skc,index_y,region_y,Positives_nkc
0,786,sKC,2020-02-01,134.142857,0,nKC,0.100000
1,787,sKC,2020-02-02,123.642857,1,nKC,0.100000
2,788,sKC,2020-02-03,113.785714,2,nKC,0.100000
3,789,sKC,2020-02-04,102.785714,3,nKC,0.100000
4,790,sKC,2020-02-05,93.285714,4,nKC,0.100000
...,...,...,...,...,...,...,...
781,1567,sKC,2022-03-23,50.071429,781,nKC,136.285714
782,1568,sKC,2022-03-24,49.571429,782,nKC,140.642857
783,1569,sKC,2022-03-25,51.285714,783,nKC,145.857143
784,1570,sKC,2022-03-26,51.642857,784,nKC,148.142857


In [60]:
#setting everything up for back filling some dates where we don't have case data for since MASCOT doesnt like NA or 0s
mar6_df = merged_df[merged_df["Date"] <= "2022-03-06"]
mar6_df.index = mar6_df.Date
first_date = pd.Timestamp('2019-12-21')
end_date = pd.Timestamp('2022-03-05')
dates = pd.date_range(first_date, end_date, freq='D')
dates.name = 'date'

In [61]:
mar6_df = mar6_df.reindex(dates,  fill_value = 0.1)

In [63]:
df_np = mar6_df[['Positives_skc', 'Positives_nkc']].to_numpy()

In [64]:
#in order to fit the matrix format of MASCOT GLM, we need it to go SKC, NKC in backwards in time order
blah = df_np.tolist()
blah.reverse()
len(blah)

In [65]:
#this prints out the predictors that can be copy and pasted into the template
print(*blah, sep=' ')
